In [58]:
from scipy.io import loadmat
import numpy as np
import seaborn as sns
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import tensorflow as tf
from sklearn.metrics import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *
import sys
sys.path.append("..") # Adds higher directory to python modules path.
from data.preprocess import *
from metric.evaluate import *
data = preprocess(path='../data/raw/CD_PD.mat')

## Predict label PD diagnosis

In [49]:
data_train, data_test = split_by_patient(data)
y_test = data_test['diagnosis'].astype(np.float32)
y_train = data_train['diagnosis'].astype(np.float32)

In [54]:
age_train = data_train['age'].astype(np.float32)
age_train = age_train.reshape(1, *age_train.shape).T
gender_train = data_train['gender'].astype(np.float32)
gender_train = gender_train.reshape(1, *gender_train.shape).T

age_test = data_test['age'].astype(np.float32)
age_test = age_test.reshape(1, *age_test.shape).T
gender_test = data_test['gender'].astype(np.float32)
gender_test = gender_test.reshape(1, *gender_test.shape).T

x_train = np.concatenate((age_train,gender_train),axis=1)
x_test = np.concatenate((age_test, gender_test), axis=1)

In [55]:
x_train[:5], x_test[:5]

(array([[63.,  1.],
        [76.,  0.],
        [85.,  0.],
        [58.,  0.],
        [79.,  0.]], dtype=float32), array([[45.      ,  1.      ],
        [53.      ,  0.      ],
        [56.      ,  0.      ],
        [59.679348,  1.      ],
        [67.      ,  1.      ]], dtype=float32))

In [59]:
#optimizer
adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
#loss function
mse = MSE

In [56]:
model = Sequential()
model.add(Input(shape=(2,)))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='softmax'))

model.compile(loss='mse', optimizer=adam)
model.build()
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 100)               300       
_________________________________________________________________
dense_13 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 101       
Total params: 10,501
Trainable params: 10,501
Non-trainable params: 0
_________________________________________________________________


In [61]:
model.fit(x_train, y_train, epochs=10)

Train on 156 samples
Epoch 1/10
156/156 [==============================] - 0s 3ms/sample - loss: 0.4103
Epoch 2/10
156/156 [==============================] - 0s 53us/sample - loss: 0.4103
Epoch 3/10
156/156 [==============================] - 0s 76us/sample - loss: 0.4103
Epoch 4/10
156/156 [==============================] - 0s 51us/sample - loss: 0.4103
Epoch 5/10
156/156 [==============================] - 0s 65us/sample - loss: 0.4103
Epoch 6/10
156/156 [==============================] - 0s 77us/sample - loss: 0.4103
Epoch 7/10
156/156 [==============================] - 0s 66us/sample - loss: 0.4103
Epoch 8/10
156/156 [==============================] - 0s 71us/sample - loss: 0.4103
Epoch 9/10
156/156 [==============================] - 0s 61us/sample - loss: 0.4103
Epoch 10/10
156/156 [==============================] - 0s 63us/sample - loss: 0.4103


In [62]:
y_pred = model.predict(x_test)
evaluate(y_true, y_pred)

ValueError: Classification metrics can't handle a mix of unknown and binary targets